## Roll Rate Analysis

En esta sección, se va a utilizar *Roll Rate Analysis* con el fin de poder determinar de una manera más acorde a los datos, aquellos clientes que se pueden identificar como buenos o malos, dependiendo de cuántos días de atraso se registran en el pago de sus cuentas.

Para efectos prácticos, se va a tomar como puntos de análisis, el último mes y el mes anterior a este, así como también se va a ignorar a los clientes que para esa fecha no cuentan con un préstamo asignado.

In [1]:
import pandas as pd 
import numpy as np 

# Se actualiza el formato de números flotantes
pd.options.display.float_format = '{:,.2f}'.format

## 1. Carga de datos

In [2]:
credit = pd.read_csv('../data/credit_record.csv')
credit

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C
...,...,...,...
1048570,5150487,-25,C
1048571,5150487,-26,C
1048572,5150487,-27,C
1048573,5150487,-28,C


In [3]:
# Validamos la cantidad de ID (clientes) únicos
credit['ID'].nunique()

45985

## 2. Estructuración de dataset

Se recategorizarán los valores `X` y 'C' de la variable *STATUS*, con el fin de mantener únicamente valores numéricos

In [4]:
credit['STATUS'] = credit['STATUS'].replace({'X':-2, 'C':-1})
credit['STATUS'] = credit['STATUS'].astype('int')
credit.dtypes

ID                int64
MONTHS_BALANCE    int64
STATUS            int32
dtype: object

In [5]:
# Validamos los valores de la variable STATUS
credit['STATUS'].value_counts()

-1    442031
 0    383120
-2    209230
 1     11090
 5      1693
 2       868
 3       320
 4       223
Name: STATUS, dtype: int64

## 3. Roll Rate analysis


In [6]:
pivot_tb = pd.pivot(credit, index='ID', columns='MONTHS_BALANCE', values='STATUS').reset_index()
#pivot_tb.reset_index(inplace=True)
pivot_tb = pivot_tb[['ID', 0, -1]]
pivot_tb = pivot_tb[(pivot_tb[0].notna()) & (pivot_tb[-1].notna())]
pivot_tb = pivot_tb[(pivot_tb[0] != -2) & (pivot_tb[-1] != -2)]
pivot_tb = pivot_tb.rename(columns={'ID':'id',0:'last_month', -1:'last_month-1'})
pivot_tb

MONTHS_BALANCE,id,last_month,last_month-1
1,5001712,-1.00,-1.00
5,5001717,-1.00,-1.00
6,5001718,-1.00,-1.00
7,5001719,-1.00,-1.00
8,5001720,1.00,0.00
...,...,...,...
45973,5150475,-1.00,-1.00
45976,5150478,-1.00,-1.00
45982,5150484,-1.00,0.00
45983,5150485,0.00,0.00


In [7]:
pivot_tb['last_month'] = pivot_tb['last_month'].astype('int')
pivot_tb['last_month-1'] = pivot_tb['last_month-1'].astype('int')
pivot_tb

MONTHS_BALANCE,id,last_month,last_month-1
1,5001712,-1,-1
5,5001717,-1,-1
6,5001718,-1,-1
7,5001719,-1,-1
8,5001720,1,0
...,...,...,...
45973,5150475,-1,-1
45976,5150478,-1,-1
45982,5150484,-1,0
45983,5150485,0,0


In [8]:
# Generamos la matriz de Transición
matriz = np.zeros((7,7),float)

for i in range(-1, 6):

    for j in range(-1, 6):
        matriz[i,j] = pivot_tb.loc[(pivot_tb['last_month-1']==i) & (pivot_tb['last_month']==j),'last_month-1'].count()
        matriz[i,j] = (matriz[i,j]*100)/ pivot_tb.loc[pivot_tb['last_month-1']==i,'last_month-1'].count()

    

matriz_df = pd.DataFrame(matriz, columns=[-1,0,1,2,3,4,5], index=[-1,0,1,2,3,4,5])
matriz_df

,-1,0,1,2,3,4,5
-1,93.07,2.24,0.00,0.00,0.00,0.01,4.68
0,53.95,34.21,3.62,0.33,0.00,0.00,7.89
1,17.65,11.76,35.29,29.41,5.88,0.00,0.00
2,16.67,0.00,16.67,0.00,66.67,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,90.00,10.00
4,0.00,0.00,0.00,0.00,0.00,98.21,1.79
5,0.00,0.00,0.00,0.00,0.00,0.00,100.00


## 4. Conclusiones

Realizando un análisis *roll back* y *roll forward* se puede identificar que los cliente con un STATUS 2 en adelante cuentan con una mayor probabilidad de aumentar sus días de atraso en los pagos en relación con aquellos clientes con STATUS -1, 0 y 1.

Por lo que para el presente análisis, los clientes con estatus -1, 0, y 1 serán etiquetados como *buenos clientes* y en cambio los clientes con status 2, 3, 4 y 5 se etiquetarán como *malos clientes*.